In [ ]:
!pip install redshift-connector

In [ ]:
import redshift_connector
import logging

# logging.basicConfig(level=logging.DEBUG)

try:
    conn = redshift_connector.connect(
        host='',
        database='dev',
        user='---',
        password='-----',
        ssl=True,
        timeout=60  # Increase the timeout if the connection takes longer
    )
    print(conn)
except redshift_connector.Error as e:
    print(f"Error: {e}")


In [ ]:
# Enabling autocommit
conn.autocommit = True

# Creating a cursor object to interact with the database
cursor = conn.cursor()

# Creating the dimDate table
cursor.execute("""
CREATE TABLE "dimDate" (
    "fips" INTEGER,
    "date" TIMESTAMP,
    "year" INTEGER,
    "month" INTEGER,
    "day_of_week" INTEGER
)
""")


cursor.execute("""
CREATE TABLE "dimHospital" (
  "fips" REAL,
  "state_name" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "hq_address" TEXT,
  "hospital_name" TEXT,
  "hospital_type" TEXT,
  "hq_city" TEXT,
  "hq_state" TEXT
)
""")



In [ ]:
# Enabling autocommit
conn.autocommit = True

# Creating a cursor object to interact with the database
cursor = conn.cursor()

# Creating the dimDate table
cursor.execute("""
CREATE TABLE "factCovid" (
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "confirmed" REAL,
  "deaths" REAL,
  "recovered" REAL,
  "active" REAL,
  "date" INTEGER,
  "positive" INTEGER,
  "negative" REAL,
  "hospitalized_currently" REAL,
  "hospitalized" REAL,
  "hospitalized_discharged" REAL
)
""")

# Creating the dimRegion table
cursor.execute("""
CREATE TABLE "dimRegion" (
  "fips" REAL,
  "province_state" TEXT,
  "country_region" TEXT,
  "latitude" REAL,
  "longitude" REAL,
  "county" TEXT,
  "state" TEXT
)
""")



In [ ]:
cursor.execute("""
copy factcovid from 's3://aws-session-datawarehousing-output/factCovid.csv'
credentials 'aws_iam_role=arn:aws:iam::975050334693:role/service-role/AmazonRedshift-CommandsAccessRole-20240625T125922'
delimiter ','
region 'us-east-2'
IGNOREHEADER 1;
""")

cursor.execute("""
copy dimregion from 's3://aws-session-datawarehousing-output/dimRegion.csv'
credentials 'aws_iam_role=arn:aws:iam::975050334693:role/service-role/AmazonRedshift-CommandsAccessRole-20240625T125922'
delimiter ','
region 'us-east-2'
IGNOREHEADER 1;
""")


cursor.execute("""
copy dimhospital from 's3://aws-session-datawarehousing-output/dimHospital.csv'
credentials 'aws_iam_role=arn:aws:iam::975050334693:role/service-role/AmazonRedshift-CommandsAccessRole-20240625T125922'
delimiter ','
region 'us-east-2'
IGNOREHEADER 1;
""")


cursor.execute("""
copy dimdate from 's3://aws-session-datawarehousing-output/dimDate.csv'
credentials 'aws_iam_role=arn:aws:iam::975050334693:role/service-role/AmazonRedshift-CommandsAccessRole-20240625T125922'
delimiter ','
region 'us-east-2'
IGNOREHEADER 1;
""")